<a href="https://colab.research.google.com/github/Charlotte11b/2025TorontoHealthDatathon/blob/main/2025_Toronto_Health_Datathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Sleep-disordered breathing (SDB), including obstructive sleep apnea (OSA), is a prevalent yet underdiagnosed condition with significant health implications. This condition affects an estimated 1 billion individuals worldwide, with moderate-to-severe OSA present in approximately 425 million people. Despite its significant health implications, including increased risk of cardiovascular disease, metabolic disorders, neurocognitive impairment, and reduced quality of life, OSA remains undiagnosed in up to 80% of affected individuals. Diagnosis is often delayed due to a complex and lengthy healthcare pathway, including waiting times to see a primary care physician, specialist referrals, and polysomnography (PSG) testing. In many healthcare systems, patients may wait several months to years for a formal diagnosis, particularly in regions with limited access to sleep specialists and diagnostic facilities.

This study examines the predictive value of self-reported measures—such as demographics, comorbidities, and the STOP-BANG questionnaire—for objective sleep study metrics like the Respiratory Disturbance Index (RDI) and Apnea-Hypopnea Index (AHI). Self-reported measures offer a promising approach for screening and risk stratification, potentially reducing unnecessary polysomnography (PSG) referrals and improving diagnostic prioritization. Their predictive capability also enhances cost-effectiveness and accessibility, ensuring that high-risk individuals receive timely diagnostic testing while minimizing healthcare expenditures. Early identification of patients at risk for moderate-to-severe OSA facilitates early intervention and disease prevention, potentially reducing complications such as cardiovascular disease and cognitive decline.

While the STOP-BANG questionnaire provides a validated risk score, machine learning (ML)-based models can improve predictive accuracy by learning complex patterns in the data, dynamically weighting features, and integrating additional predictors such as body mass index (BMI), neck circumference, comorbidities, and biomarkers. ML models also enable multimodal data integration, incorporating self-reported information with wearable device data (e.g., Fitbit, Apple Watch), electronic health records (EHRs), and imaging (e.g., upper airway MRI or CT scans). Unlike traditional categorical scoring, ML models provide continuous risk estimation, allowing for personalized thresholds and more precise risk stratification. Additionally, ML models offer feature importance analysis and explainability using methods like SHAP values and decision trees, improving clinical interpretability. These models can be adapted to specific populations, addressing demographic and physiological variations, and can be automated and scaled via clinical decision support systems (CDSS) and mobile health (mHealth) applications for real-time risk assessment.

Leveraging machine learning in predictive modeling could significantly enhance clinical decision-making, optimize healthcare resource allocation, and improve access to timely OSA diagnosis and management.

In [ ]:
!pip install xgboost
!pip install shap
!pip install --upgrade scikit-learn
!pip check

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.feature_selection import mutual_info_regression, RFE, SelectFromModel
from sklearn.linear_model import LogisticRegression, LassoCV, LinearRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from xgboost import XGBClassifier, XGBRegressor
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, mean_squared_error, r2_score, mean_absolute_error
import shap
import matplotlib.pyplot as plt
from sklearn.inspection import partial_dependence
import statsmodels.api as sm

In [ ]:
# 1. Loads the dataset from a CSV file and preprocesses it
# defines target labels, encodes categorical variables, standardizes features, and splits into training/validation sets

def load_data(csv_path, target_column, feature_columns):
    # Load dataset
    df = pd.read_csv(csv_path)
    df = df[feature_columns + [target_column]].dropna()

    # Encode categorical variables
    for col in df.select_dtypes(include=['object', 'category']).columns:
        df[col] = LabelEncoder().fit_transform(df[col])

    # Split into train/validation sets
    X = df[feature_columns]
    y = df[target_column]
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)

    # Standardize numerical features
    scaler = StandardScaler()
    X_train[X_train.columns] = scaler.fit_transform(X_train)
    X_val[X_val.columns] = scaler.transform(X_val)

    return X_train, X_val, y_train, y_val

In [ ]:
# Specify the CSV path, target column, and feature columns
csv_path = "/home/jupyter/gcs/TCAIREM_SleepLabData.csv"
target_column = "Slpahi"
feature_columns = [
    'SBsnore', 'SBtired', 'SBObs', 'SBbp', 'SBneck',
    'Sex', 'BMI', 'Height', 'Weight'
]

# Load the data
X_train, X_val, y_train, y_val = load_data(csv_path, target_column, feature_columns)

In [ ]:
def feature_selection(X_train, y_train):
    # Mutual Information for Regression
    mi_scores = mutual_info_regression(X_train, y_train)
    mi_selected = X_train.columns[mi_scores > np.median(mi_scores)]

    # Recursive Feature Elimination (RFE) with a Regression Model
    model = LinearRegression()  # Use a regression model here
    rfe = RFE(model, n_features_to_select=int(len(X_train.columns) / 2))
    rfe.fit(X_train, y_train)
    rfe_selected = X_train.columns[rfe.support_]

    # LASSO Feature Selection
    lasso = LassoCV(cv=5).fit(X_train, y_train)
    lasso_selected = X_train.columns[np.abs(lasso.coef_) > 0]

    # XGBoost Feature Selection
    xgboost_model = xgb.XGBRegressor(objective='reg:squarederror')
    xgboost_model.fit(X_train, y_train)
    xgboost_importance = xgboost_model.feature_importances_
    xgboost_selected = X_train.columns[xgboost_importance > np.median(xgboost_importance)]

    # SHAP Feature Selection
    explainer = shap.Explainer(xgboost_model, X_train)
    shap_values = explainer(X_train)
    shap_importance = np.abs(shap_values.values).mean(axis=0)
    shap_selected = X_train.columns[shap_importance > np.median(shap_importance)]

    # Union of selected features using OR
    selected_features = set(mi_selected) | set(rfe_selected) | set(lasso_selected) | set(xgboost_selected) | set(shap_selected)

    print("Selected Features from each method:")
    print("Mutual Information:", list(mi_selected))
    print("RFE:", list(rfe_selected))
    print("LASSO:", list(lasso_selected))
    print("XGBoost:", list(xgboost_selected))
    print("SHAP:", list(shap_selected))
    print("Final selected features:", list(selected_features))

    return list(selected_features)

selected_features = feature_selection(X_train, y_train)

In [ ]:
def train_models(X_train, y_train, X_val, y_val):
    # Use regression models
    models = {
        "Linear Regression": LinearRegression(),
        "Random Forest": RandomForestRegressor(n_estimators=100),
        "XGBoost": XGBRegressor(objective='reg:squarederror')
    }

    results = {}
    for name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)

        # Use regression metrics
        mse = mean_squared_error(y_val, y_pred)
        mae = mean_absolute_error(y_val, y_pred)
        r2 = r2_score(y_val, y_pred)

        results[name] = {"MSE": mse, "MAE": mae, "R2 Score": r2}

    # Find the best model based on R2 score
    best_model_name = max(results, key=lambda k: results[k]['R2 Score'])
    best_model = models[best_model_name]

    print("Model Performance:")
    print(pd.DataFrame(results).T)
    print(f"Best Model: {best_model_name}")

    return best_model, best_model_name

X_train, X_val = X_train[selected_features], X_val[selected_features]
best_model, best_model_name = train_models(X_train, y_train, X_val, y_val)


In [ ]:
# convert y (AHI score) into a categorical variable
def convert_to_categories(y):
    # Define categories based on the provided ranges
    bins = [0, 5, 15, 30, float('inf')]  # Define bins for risk levels
    labels = ['Normal', 'Low Risk', 'Moderate Risk', 'High Risk']
    y_cat = pd.cut(y, bins=bins, labels=labels, right=False)
    return y_cat

In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

def train_and_evaluate_classifiers(X_train, y_train, X_val, y_val):
    # Model Initialization
    models = {
        'Logistic Regression': LogisticRegression(max_iter=1000, multi_class='ovr'),
        'Random Forest': RandomForestClassifier(),
        'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
    }

    # Store performance metrics
    performance_metrics = {}

    # Train models and evaluate
    for model_name, model in models.items():
        # Train the model
        model.fit(X_train, y_train)

        # Make predictions
        y_pred = model.predict(X_val)

        # Calculate metrics
        acc = accuracy_score(y_val, y_pred)
        prec = precision_score(y_val, y_pred, average='weighted')
        sens = recall_score(y_val, y_pred, average='weighted')

        # Confusion matrix to calculate specificity
        cm = confusion_matrix(y_val, y_pred)
        specificity = np.diag(cm) / cm.sum(axis=1)  # Specificity per class
        avg_specificity = np.mean(specificity)

        # Store results
        performance_metrics[model_name] = {
            'Accuracy': acc,
            'Precision': prec,
            'Sensitivity': sens,
            'Specificity': avg_specificity
        }

    # Convert to DataFrame for easy comparison
    performance_df = pd.DataFrame(performance_metrics).T
    print(performance_df)

    # Select best model based on accuracy
    best_model_name = performance_df['Accuracy'].idxmax()
    best_model = models[best_model_name]
    print(f'\nBest Model: {best_model_name}')

    return performance_df, best_model_name


In [ ]:
# Prepare the data: Convert y_train and y_val into categorical variables
y_train_cat = convert_to_categories(y_train)
y_val_cat = convert_to_categories(y_val)

# Encode categorical labels as numeric labels
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train_cat)
y_val_encoded = le.transform(y_val_cat)

# Select the features based on feature selection (use the selected features)
X_train, X_val = X_train[selected_features], X_val[selected_features]

# Train and evaluate models
performance_df, best_model_name = train_and_evaluate_classifiers(X_train, y_train_encoded, X_val, y_val_encoded)

In [ ]:
def create_nomogram(X_train, y_train, X_val, y_val):
    # Fit a linear regression model
    model = LinearRegression()
    model.fit(X_train, y_train)

    # Predict on the validation set
    y_pred = model.predict(X_val)

    # Mean squared error on validation set
    mse = mean_squared_error(y_val, y_pred)
    print(f'Mean Squared Error: {mse}')

    # Coefficients of the linear regression model
    coefficients = model.coef_

    # Create a nomogram-like plot
    plt.figure(figsize=(10, 6))

    # Plot each feature's influence on the target
    for i, coef in enumerate(coefficients):
        # Make sure the feature name is correct
        feature_name = X_train.columns[i]  # Get the feature name from the columns of X_train
        plt.plot(X_train.iloc[:, i], X_train.iloc[:, i] * coef, label=f"{feature_name} * Coef({coef:.2f})")

    # Adding plot aesthetics
    plt.axhline(0, color='gray', lw=1)
    plt.xlabel('Feature Values')
    plt.ylabel('Influence on Target')
    plt.title('Nomogram-like Visualization for Linear Regression Model')
    plt.legend()
    plt.show()

    return model

# Example usage with your data:
model = create_nomogram(X_train, y_train, X_val, y_val)
